In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

warnings.filterwarnings("ignore")

# Configurações de visualização
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

In [ ]:
# Caminho do arquivo de entrada (CSV agregado)
input_csv_path = "furtos_agregado_por_cidades2.csv"

# Diretório para salvar gráficos
output_plot_dir = "eda/aggreg"
if not os.path.exists(output_plot_dir):
    os.makedirs(output_plot_dir)
    print(f"Diretório de gráficos criado em: {output_plot_dir}")

def run_eda_on_aggregated_data(csv_path, plot_dir):
    """Executa a EDA no arquivo CSV agregado e salva os gráficos."""
    print(f"--- Iniciando EDA para {csv_path} ---")

    try:
        df_agg = pd.read_csv(csv_path, delimiter=';')
        print(f"Dados agregados carregados. Shape: {df_agg.shape}")
    except FileNotFoundError:
        print(f"ERRO: Arquivo não encontrado em {csv_path}")
        return False
    except Exception as e:
        print(f"Erro ao carregar o arquivo CSV: {e}")
        return False

    print("\nPrimeiras 5 linhas dos dados agregados:")
    print(df_agg.head())

    print("\nInformações das colunas e tipos de dados:")
    df_agg.info()

    print("\nVerificação de valores ausentes:")
    print(df_agg.isnull().sum())
    df_agg.dropna(inplace=True)
    print(f"Shape após remover NaNs (se houver): {df_agg.shape}")


    df_agg['Area'] = df_agg['Area'].str.replace(',','.',regex=True).astype(float)
    
    # Colunas numéricas para análise
    numeric_cols = ["População", "Area", "Densidade_Recalculada", "Total"]

    print("\n--- 2. Análise Descritiva ---")
    print("Estatísticas Descritivas:")
    print(df_agg.describe())

    print("\n--- 3. Visualizações Exploratórias ---")

    print("Gerando histogramas das distribuições...")
    df_agg[numeric_cols].hist(bins=30, figsize=(15, 10))
    print(df_agg["Area"])
    
    plt.tight_layout()
    hist_path = os.path.join(plot_dir, "aggregated_numeric_distributions.png")
    try:
        plt.savefig(hist_path)
        print(f"Histogramas salvos em: {hist_path}")
    except Exception as e:
        print(f"Erro ao salvar histogramas: {e}")
    plt.close()

    print("Gerando gráficos de dispersão (Total vs Outras Métricas)...")
    scatter_metrics = ["População", "Area", "Densidade_Recalculada"]
    for metric in scatter_metrics:
        plt.figure(figsize=(10, 6))
        sns.scatterplot(data=df_agg, x=metric, y="Total")
        plt.title(f"Relação entre Total de Furtos e {metric}")
        plt.xlabel(metric)
        plt.ylabel("Total de Furtos Agregado por Cidade")
        scatter_path = os.path.join(plot_dir, f"scatter_total_vs_{metric.lower()}.png")
        try:
            plt.savefig(scatter_path)
            print(f"Gráfico Total vs {metric} salvo em: {scatter_path}")
        except Exception as e:
            print(f"Erro ao salvar gráfico Total vs {metric}: {e}")
        plt.close()

    print("\n--- 4. Análise de Correlação ---")
    correlation_matrix = df_agg[numeric_cols].corr()

    print("Matriz de Correlação:")
    print(correlation_matrix)

    print("\nGerando heatmap de correlação...")
    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Heatmap de Correlação (Dados Agregados por Cidade)")
    heatmap_path = os.path.join(plot_dir, "aggregated_correlation_heatmap.png")
    try:
        plt.savefig(heatmap_path)
        print(f"Heatmap salvo em: {heatmap_path}")
    except Exception as e:
        print(f"Erro ao salvar heatmap: {e}")
    plt.close()

    print("\n--- EDA Concluída ---")
    return True

In [ ]:
    success = run_eda_on_aggregated_data(input_csv_path, output_plot_dir)
    if success:
        print(f"\nScript de EDA concluído. Gráficos salvos em {output_plot_dir}")
    else:
        print("\nScript de EDA falhou.")

